In [1]:
from azure.storage.blob import BlobServiceClient
import logging
from datetime import datetime,timedelta
import pytz
import boto3
from botocore.exceptions import ClientError
import os
import pandas as pd


container_name = "enelx"
today = datetime.utcnow()
yesterday = today - timedelta(days=1)
dateRef = today.strftime('%Y%m%d')
thresholdTime = pytz.UTC.localize(yesterday)
localPath='/Users/s.eromonsei/my_sandbox/Engineering/DataEngineering/LocalStack/src'
bucketpath= "my-bucket"

#bblobConn=WasbHook(wasb_conn_id="AzureBlob",public_read=False)

blob_service_client =  BlobServiceClient.from_connection_string('BlobEndpoint=https://blobendpoint;')


logging.basicConfig(filename='new.log', level=logging.DEBUG,
                    format='(name)s:%(asctime)s:%(levelname)s:%(message)s')

def getbloblist(container_name):
        """Getting Blob based on the date criteria
        Parameters
        - params container_name (str) – The name of the container
        - Params prefix (str | None) – Filters the results to return only blobs whose names begin with the specified prefix
        """
        my_container = blob_service_client.get_container_client(container_name)
        my_blobs = my_container.list_blobs(name_starts_with='MeterData/uploads/')
        # Instantiate a BlobServiceClient using a connection string
        for blob in my_blobs:
            # write blob to local directory
             if thresholdTime < blob.creation_time:
                print(blob)
                num = blob.name.count('/')
                blobName = blob.name.split('/')[num]
                dateRef = blob.creation_time.strftime('%Y%m%d')
                bytes = my_container.get_blob_client(blob).download_blob().readall()
                download_file_path = os.path.join(localPath + dateRef, blobName)
                print(download_file_path)
                os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
                with open(download_file_path, 'wb') as file:
                    file.write(bytes)

                    file.close()
                df=pd.read_csv(download_file_path)
                print(df)
                # Write to s3 bucket
                file_name=download_file_path
                key='caiso/battery/enel-x/'+blobName
                print(key)
                #bucket_name= bucketpath
                Bucket= bucketpath
                s3_connection = boto3.client('s3', aws_access_key_id='',aws_secret_access_key='',endpoint_url='http://localhost:4566')
                print(f'connection {s3_connection} was succesful')
                try:
                        #response = s3_connection.upload_file(file_name, bucket_name, object_Name)
                        #s3_connection.put_object(Body=file_name, Bucket=Bucket,Key=key)
                        with open(file_name, 'rb') as data:
                                s3_connection.upload_fileobj(data,Bucket=Bucket, Key=key)
                                logging.info("Upload successful")
                except ClientError as e:
                        logging.info(e,"Failed connection and upload")
        return logging.info("Blobs successful")

getbloblist(container_name)


{'name': 'MeterData/uploads/EnelX_RP1_5min_interval_data_for_2022-12-20_on_2022-12-21_1300.csv', 'container': 'enelx', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2022, 12, 21, 18, 0, 57, tzinfo=datetime.timezone.utc), 'etag': '0x8DAE37D4FEDAAE6', 'size': 21287, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'text/csv', 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, '